# Pandas and Hierarchical Indexing #

## Context
Pandas' ability to index data offers additional power to the way you work with data. More interesting is pandas' hierarchical indexing feature, it allows you to slice and dice data in convenient ways.

Python and pandas allow you to manage data more efficiently and effectively than, say, Stata. One important problem with Stata is that you have all your data in one large file. As a result your work-file tends to grow, and thus gets messy.

## Efficient data management
Python and pandas store data in various ways, e.g. in lists, tuples, sets, dictionaries, DataFrames and Series. This is super efficient: each data item can be stored in its most efficient form.

On top of that, for pandas there is **indexing** and **hierarchical indexing**. These features offers you the ability to focus on specific data sets within a single DataFrame. For example, your DataFrame may contain firm identification information, such as names and permcos, adjacent to numerical data. If you want to analyze the numbers, items such as names and permcos stand in the way. Hierarchical Indexing offers you a solution: you can set an index in such a way that your analysis only examines the numbers and ignores names and other non-numerical data.

Indexing also allows you to **quickly produce tables**, (which then can be used as new DataFrames, etc).

An additional feature of indexing is that you can use it to **merge** files quickly. 

## Examples 
The examples below demonstrate the virtues of indexing and hierarchical indexing. 

To reproduce the examples, please download from WRDS some annual and quarterly data from the CRSP Compustat Merge (CCM) database. I use csv format data.

More specifically: the **annual data** contains a history (1970-2015) of the following variables:

In [ ]:
datadate, gvkey, curcd, conm, fyr, exchg, cik, costat, fic, sic, fyear, at, caps, ceq,  csho, dvc, dvp, ni

The **quarterly data** contains a history (1970-2015) of the following variables:

In [ ]:
datadate, gvkey, prccq, rdq, cshoq, fyr, fyearq

## Example 3, slicing and Hierarchical Indexing to report descriptives and to select sets of columns:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
try:
    os.chdir('/home/martien/Dropbox/../folder/')
except Exception:
    os.chdir('C:/Users/martien/Dropbox/../folder/')
pd.set_option('display.max_columns', None)

In [ ]:
def make_date(df, lyst):
    for item in lyst:
        df[item] = pd.to_datetime(df[item], format='%Y%m%d')
    return(df)

def intersect(a, b):
    """ return the intersection of two lists """
    return list(set(a) & set(b))

def read_compustat(fn, lyst, date_lyst=['datadate']):
    df = pd.read_csv(fn, low_memory=False)
    df.columns = df.columns.str.lower()
    # to prevent from trying to load data from columns that are not on file:
    lyst = intersect(lyst,list(df))
    df = df.drop_duplicates(subset=['gvkey', 'datadate'])
    make_date(df, date_lyst)
    return(df[lyst])

In [ ]:
typlist = ['id',    'id',   'id',  'id',    'id',  'id',     'id',  'id',   'data',  'data', 'data', 'data', 'data', 'data', 'data', 'data', 'data',  'data']
varlist = ['curcd', 'conm', 'fyr', 'exchg', 'cik', 'costat', 'fic', 'sic',  'fyear', 'at',   'caps', 'ceq',  'csho', 'dvc',  'dvp',  'ni',   'sicno', 'roe']

dfa =  read_compustat('annual.csv', ['gvkey', 'datadate'] + varlist, ['datadate'])
dfa = dfa.loc[(dfa.curcd == 'USD') & 
              (dfa.fic == 'USA') & 
              (~pd.isnull(dfa.ni)) & 
              (~pd.isnull(dfa.ceq)) & 
              (~pd.isnull(dfa.fyr)) &  
              (~pd.isnull(dfa.fyear))]

dfa[['fyr', 'fyear']] = dfa[['fyr', 'fyear']].astype(int)

Make a simple industry classification, calculate ROE, where ROE is within reasonable bounds:

In [ ]:
dfa['sicno'] = np.floor_divide(dfa.sic.fillna(0), 1000).astype(int)
dfa['roe'] = dfa.ni.div(dfa.ceq).clip(-1,10)
dfa.head(2)

In [ ]:
dfa.set_index(['gvkey', 'datadate'],inplace=True)
dfa.head(2)

Read the quarterly data and calculate market value:

In [ ]:
dfq = read_compustat('quarterly.csv', ['datadate', 'gvkey', 'prccq', 'rdq', 'cshoq', 'fyr', 'fyearq'], ['datadate', 'rdq'])
dfq['mv'] = dfq.prccq * dfq.cshoq

Set the index and join.

In [ ]:
dfq.set_index(['gvkey', 'datadate'],inplace=True)
dfa=dfa[varlist]
dfa = dfa.join(dfq['mv'])

### Slicing and Hierarchical Indexing - descriptive statistics. ###
We will use Hierarchical Indexing to generate descriptive statistics.

In [ ]:
dfa.reset_index(inplace=True)
dfa.set_index(['sicno', 'fyear'],inplace=True)
dfa.head(2)

In [ ]:
# Industry statistics: means for three variables
dfa[['ceq', 'at', 'roe']].mean(level='sicno')

Trends over time, but we do not want to report each year. So lets examine every 5th year

In [ ]:
yrs = list(range(1971, 2015, 5))
print(yrs)

In [ ]:
# Trends over time:
dfa.loc(axis=0)[:,yrs][['ceq', 'at', 'roe']].mean(level='fyear')

In [ ]:
#Comprehensive statistics:
dfa[['ceq', 'at', 'roe']].describe()

Below we use the pivot function to create a neat cross table for medians of ROE for years and industries

In [ ]:
my_cross_table = dfa.reset_index().groupby(['sicno', 'fyear'])[['roe']].median().reset_index().pivot(index='fyear', columns='sicno', values='roe')
my_cross_table.loc[yrs]

### Slicing and Hierarchical Indexing - Sets of columns. ###
Now we will use a the ypelist and varlist from above to group columns.

But first we will reset and set the index.

In [ ]:
dfa.reset_index(inplace=True)
dfa.set_index(['gvkey', 'datadate'],inplace=True)
dfa.head(2)

I forgot to add market value (a data variable) to the list of variables. So let's do that now:

In [ ]:
varlist += ['mv']
typlist += ['data']

In [ ]:
# Check if both list are equally long:
len(typlist)
dfa.shape
len(varlist)

In [ ]:
# Check if both are in sync:
print(varlist)
print(typlist)

In [ ]:
# To make sure the data frame has all variables in the order of the varlist.
dfa=dfa[varlist].copy()
dfa.head(1)
#list(dfa)

Here we add the typlist as an additional column index. 

As you can see below, there are two column indexes: 'id' and 'data'. This enbales us to examine the data section separately.

In [ ]:
#dfa.columns=[varlist]
dfa.columns=[typlist, varlist]
dfa.columns.names = ['type','compustat']
dfa.head()

In [ ]:
# show firm ID info
dfa['id'].head(2)
dfa['id'].tail(2)

In [ ]:
# show data only
dfa['data'].head(2)
dfa['data'].tail(2)

The next examples show some serious pandas magic: I only report descriptive info of the 'data' columns:

In [ ]:
dfa['data'].describe()

In [ ]:
# Subset: Total Assets only
dfa['data']['at'].describe()

In [ ]:
# Only for a set date or period
dfa.loc(axis=0)[:,'1974-12-31']['data'].describe()
dfa.loc(axis=0)[:,'1974']['data'].describe()

Examine slices of data

In [ ]:
# Post GFC
dfa.loc(axis=0)[:,'2009':'2016']['data'].describe()
# Microsoft
dfa.loc(axis=0)[12141,'1984':'2010']['data'].describe()